In [1]:
import numpy as np
# https://medium.com/@stevehiehn/how-to-generate-music-with-python-the-basics-62e8ea9b99a5
from midiutil import MIDIFile
from mingus.core import chords
from IPython.display import Audio
import os
from collections import defaultdict

In [2]:
stats = defaultdict(list)

In [3]:
NOTES = ['C', 'C#', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'Ab', 'A', 'Bb', 'B']
# play one chord per range, where the range chooses the interval of the lowest note in the chord
# RANGES = [
#     range(35, 55),
#     range(55, 75),
# ]
RANGES = [range(40, 60)]
MAJORMINOR = ['M', 'm']
SEVENTH = [''] # TODO add 7

def swap_accidentals(note):
    if note == 'Cb':
        return 'B'
    if note == 'Db':
        return 'C#'
    if note == 'D#':
        return 'Eb'
    if note == 'E#':
        return 'F'
    if note == 'Gb':
        return 'F#'
    if note == 'G#':
        return 'Ab'
    if note == 'A#':
        return 'Bb'
    if note == 'B#':
        return 'C'

    return note
def note_to_number(note):
    note = swap_accidentals(note)
    return NOTES.index(note)

track = 0
channel = 0
time = 0  # In beats
duration = 2  # In beats
tempo = 60  # In BPM
volume = 100  # 0-127, as per the MIDI standard
n_repeats = 6

In [31]:
def generate_midi():
    base = np.random.choice(NOTES)
    majorminor = np.random.choice(MAJORMINOR)
    seventh = np.random.choice(SEVENTH)
    chord = base + majorminor + seventh

    MyMIDI = MIDIFile(1)  # One track, defaults to format 1 (tempo track is created
    # automatically)
    MyMIDI.addTempo(track, time, tempo)

    notes = chords.from_shorthand(chord)
    # print(chord, notenums)
    for i in range(n_repeats):
        notenums = []
        for baserange in RANGES:
            base = np.random.choice(baserange)
            baseC = (base // 12) * 12
            basenote = base % 12
            for note in notes:
                notenum = note_to_number(note)
                if notenum < basenote:
                    notenums.append(baseC + notenum + 12)
                else:
                    notenums.append(baseC + notenum)
        for j, notenum in enumerate(notenums):
            MyMIDI.addNote(track, channel,
                           notenum, i * duration,
                           duration + (2 if i == (n_repeats - 1) else 0), volume)
    with open('out.mid', 'wb') as output_file:
        MyMIDI.writeFile(output_file)
    os.system('timidity out.mid')
    return chord


In [36]:
chord = generate_midi()
Audio('out.ogg', autoplay=True)

Playing out.mid
MIDI file: out.mid
Format: 1  Tracks: 2  Divisions: 960
Output out.ogg
Playing time: ~18 seconds
Notes cut: 0
Notes lost totally: 0


In [37]:
guess = input('Guess: ')
n_guess = 1
def filter(guess):
    return swap_accidentals(guess[ :2]) + guess[2: ]
while filter(guess) != chord:
    if guess == 'break' or n_guess >= 6:
        stats[chord].append(6)
        print('Gave up, it was', chord)
        break
    elif guess != 're':
        print('WRONG')
        n_guess += 1
    else:
        print('Undoing last guess')
        n_guess -= 1

    guess = input('Guess: ')
else:
    print('Yes, it was %s, took %d tries' % (chord, n_guess))
    stats[chord].append(n_guess)
print('\nCurrent stats are:')
for k, v in stats.items():
    print('%s(%.1f):\t%s' % (k, np.mean(v), v))

Guess: F#M
Yes, it was F#M, took 1 tries

Current stats are:
AbM(1.0):	[1]
Bm(1.0):	[1, 1, 1]
C#M(1.0):	[1, 1]
CM(1.0):	[1]
C#m(1.0):	[1]
GM(1.0):	[1]
Am(1.0):	[1, 1]
EM(1.0):	[1]
DM(1.0):	[1]
F#M(1.5):	[2, 1]
EbM(1.0):	[1]
